<a href="https://colab.research.google.com/github/kk-ddfafafa/created/blob/master/deep_rl_trader_STOCK_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# coding:utf-8
import gym
import numpy as np
import pandas as pd
import random
import os
import pdb

import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
import tensorflow as tf

ModuleNotFoundError: ignored

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving process_N225.csv to process_N225.csv


In [ ]:
import io
df = pd.read_csv(io.StringIO(uploaded['process_N225.csv'].decode('utf-8')), header=0,index_col = 0)
#df.drop(['Adj Close'], 1, inplace=True)
#df = df.iloc[:,[0,1,2,4,3]]

In [ ]:
df =  df.fillna(method = 'bfill')

In [ ]:
n = df.shape[0]
p = df.shape[1]

train_start=0
train_end = int(np.floor(0.8*n))
test_start = train_end + 1
test_end = n
data_train = df.iloc[:train_end,:]
data_test = df.iloc[test_start:test_end,:]

data_train = data_train.values 
data_test = data_test.values


# データの正規化
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(data_train)
data_train_norm = scaler.transform(data_train)
data_test_norm = scaler.transform(data_test)

In [ ]:
train = data_train_norm
test = data_test_norm
data = train
data_train[0:74,:].shape

(74, 7)

In [ ]:
import gym
from gym import spaces
from gym.utils import seeding
import math

# position constant
LONG = 0
SHORT = 1
FLAT = 2

# action constant
BUY = 0
SELL = 1
HOLD = 2

class Env(gym.Env):
    
    def __init__(self,window_size,show_trade=True):
        self.show_trade = show_trade
        self.actions = ["LONG", "SHORT", "FLAT"]
        self.fee = 500
        self.seed()
        self.file_list = []
        self.df =data
        
        # n_features
        self.window_size = window_size
        self.n_features = self.df.shape[1]
        self.shape = (self.window_size, self.n_features+4)
        
        # defines action space
        self.action_space = spaces.Discrete(len(self.actions))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=self.shape, dtype=np.float32)

    def render(self, mode='human', verbose=False):
        return None

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]


    def step(self, action):

        if self.done:
            return self.state, self.reward, self.done, {}
        self.reward = 0
        
        self.action = HOLD  # hold
        
        if action == BUY: # buy
            if self.position == FLAT: # if previous position was flat
                self.position = LONG # update position to long
                self.action = BUY # record action as buy
                self.entry_price = self.closingPrice # maintain entry price
                self.wallet -= self.entry_price*10
                self.entry_tick = 0
#                print(self.entry_price,self.position)
            elif self.position == SHORT: # if previous position was short
                self.position = FLAT  # update position to flat
                self.action = BUY # record action as buy
                self.exit_price = self.closingPrice
                self.reward = self.entry_price - self.exit_price - self.fee
                self.wallet += self.closingPrice*10
                self.cumulative_return += self.reward 
                self.entry_price = 0 # clear entry price
                self.n_short += 1 # record number of short
#                print(str(self.closingPrice),str(self.reward),str(self.wallet))
        elif action == 1: # vice versa for short trade
            if self.position == FLAT:
                self.position = SHORT
                self.action = 1
                self.entry_price = self.closingPrice
                self.wallet -= self.entry_price*10
                self.entry_tick = 0
#                print(self.entry_price,self.position)
            elif self.position == LONG:
                self.position = FLAT
                self.action = 1
                self.exit_price = self.closingPrice
                self.reward = self.exit_price - self.entry_price - self.fee
                self.wallet += self.closingPrice*10
                self.cumulative_return += self.reward
                self.entry_price = 0
                self.n_long += 1
#                print(str(self.closingPrice),str(self.reward),str(self.wallet))
        

        
        if(self.position == LONG):
            temp_reward = self.closingPrice - self.entry_price - self.fee
            new_portfolio = self.wallet + self.closingPrice*10
        elif(self.position == SHORT):
            temp_reward = self.entry_price - self.closingPrice - self.fee
            new_portfolio = self.wallet + self.closingPrice*10
        else:
            temp_reward = 0
            new_portfolio = self.wallet

        self.portfolio = new_portfolio
        self.current_tick += 1
        self.entry_tick += 1
        if(self.show_trade and self.current_tick%100 == 0):
            print("Tick: {0}/ Wallet: {1}".format(self.current_tick, self.portfolio))
            print("Long: {0}/ Short: {1}".format(self.n_long, self.n_short))
        self.history.append((self.action, self.current_tick, self.closingPrice, self.portfolio, self.reward))
        self.updateState()

#        if self.portfolio < 0:
#            self.done = True;self.reward -= 1000000
#        print(self.portfolio,"                          ")
        if self.reward > 0:
            self.reward = 1 + self.reward/self.fee
        elif self.reward < 0:
            self.reward = -1
        if self.current_tick >= (len(data_train[:,0]) -1):
            self.done = True
            #self.reward = self.get_profit()
            if self.portfolio > 500000:
                self.reward = 10
        return self.state, self.reward, self.done, {'portfolio':np.array([self.portfolio])}

    def get_profit(self):
        if(self.position == LONG):
            profit = (self.closingPrice - self.entry_price - self.fee)*10
        elif(self.position == SHORT):
            profit = (self.entry_price - self.closingPrice - self.fee)*10
        else:
            profit = 0
        return profit


    def reset(self):
        # self.current_tick = random.randint(0, self.df.shape[0]-1000)
        self.current_tick = 0
#        print("start episode ... {0} at {1}" .format(self.rand_episode, self.current_tick))

        # positions
        self.n_long = 0
        self.n_short = 0

        # clear internal variables
        self.history = [] # keep buy, sell, hold action history
        self.cumulative_return = 0 # initial balance, u can change it to whatever u like
        self.portfolio = float(self.cumulative_return) 
        self.profit = 0
        self.wallet = 500000
        self.entry_tick = 0# Forced settlement 

        
        self.action = HOLD
        self.position = FLAT
        self.done = False

        self.updateState() # returns observed_features +  opened position(LONG/SHORT/FLAT) + profit_earned(during opened position)
        print("リセットしました",self.current_tick)
        return self.state

    def updateState(self):
        def one_hot_encode(x, n_classes):
            return np.eye(n_classes)[x]
        self.closingPrices = data_train[:,0]
        self.closingPrice = float(self.closingPrices[self.current_tick])
        prev_position = self.position
        one_hot_position = one_hot_encode(prev_position,3)
        profit = self.get_profit()
        # append two
        self.state = np.concatenate((self.df[self.current_tick], one_hot_position, [profit]))
        return self.state

In [ ]:
!pip install keras-rl

     |████████████████████████████████| 40kB 6.5MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48379 sha256=d06e3212468e9fdd2a3bd16bee671039e8f9bedfaf0728db2f239263d80597f0
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl


In [ ]:
import numpy as np
from rl.core import Processor
from rl.util import WhiteningNormalizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler

ADDITIONAL_STATE = 4
class NormalizerProcessor(Processor):
    def __init__(self):
        self.scaler = StandardScaler()
        self.normalizer = None

    def process_state_batch(self, batch):
        batch_len = batch.shape[0]
        k = []
        for i in range(batch_len):
            observe = batch[i][..., :-ADDITIONAL_STATE]
            observe = self.scaler.fit_transform(observe)
            agent_state = batch[i][..., -ADDITIONAL_STATE:]
            temp = np.concatenate((observe, agent_state),axis=1)
            temp = temp.reshape((1,) + temp.shape)
            k.append(temp)
        batch = np.concatenate(tuple(k))
        return batch

Using TensorFlow backend.


In [ ]:
# import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, CuDNNLSTM
from keras.optimizers import Adam
from keras.models import load_model
# keras-rl agent
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory


def create_model(shape, nb_actions):
    model = Sequential()
    model.add(CuDNNLSTM(256, input_shape=shape, return_sequences=True))
    model.add(CuDNNLSTM(256))
    model.add(Dense(128))
    model.add(Activation('relu'))
    model.add(Dense(nb_actions, activation='linear'))
    return model

def main():
    # OPTIONS
    ENV_NAME = Env(75)
    env = Env(75)
    TIME_STEP = 75


    # random seed
#    np.random.seed(123)
#    env.seed(123)

    nb_actions = env.action_space.n
    model = create_model(shape=env.shape, nb_actions=nb_actions)
    print(model.summary())

    # Finally, we configure and compile our agent. You can use every built-in Keras optimizer and even the metrics!
    memory = SequentialMemory(limit=50000, window_length=TIME_STEP)
    # policy = BoltzmannQPolicy()
    policy = EpsGreedyQPolicy()
    # enable the dueling network
    # you can specify the dueling_type to one of {'avg','max','naive'}
    dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=200,
                   enable_dueling_network=True, dueling_type='avg', target_model_update=1e-2, policy=policy,
                   processor=NormalizerProcessor())
    dqn.compile(Adam(lr=1e-3), metrics=['mae'])

    print("train start")
#    dqn.load_weights('dqn_{}_weights.h5f'.format(ENV_NAME))
    # train
    dqn.fit(env, nb_steps=50000, nb_max_episode_steps=(len(data_train[:,0])), visualize=False, verbose=2)
    dqn.save_weights('dqn_{}_weights.h5f'.format(ENV_NAME), overwrite=True)

if __name__ == '__main__':
    main()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cu_dnnlstm_3 (CuDNNLSTM)     (None, 75, 256)           275456    
_________________________________________________________________
cu_dnnlstm_4 (CuDNNLSTM)     (None, 256)               526336    
_________________________________________________________________
dense_4 (Dense)              (None, 128)               32896     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 387       
Total params: 835,075
Trainable params: 835,075
Non-trainable params: 0
_________________________________________________________________
None
train start
Training for 50000 steps ...
リセットしました 0
Tick: 100/ Wallet: 517481.79689999996
Long: 1/ Short: 0
T